In [1]:
import pandas as pd

In [2]:
lut = pd.read_pickle('lang_uri_time_wb_avail_all.pkl.gz', compression='gzip')
macd_all = pd.read_pickle('data/macd_all.pkl.gz', compression='gzip')
lut['uri_unsh_no_clean'] = lut.uri_unshrtn_raw.combine_first(lut.uri_unshrtn_bitly.combine_first(lut.uri_unshrtn.combine_first(lut.uri)))
lut['uri_mem_avail'] = lut.uri.map(~macd_all.drop_duplicates(subset='uri').set_index('uri').first_snap.isna()).fillna(False)
lut['uri_unsh_no_clean_mem_avail'] = lut.uri_unsh_no_clean.map(~macd_all.drop_duplicates(subset='uri').set_index('uri').first_snap.isna()).fillna(False)
curl = pd.read_csv('data/curl_results_parsed.csv.gz', compression='gzip')
lut['status_code'] = lut.uri_unsh_no_clean.map(curl.drop_duplicates(subset='url').set_index('url').status_code)
lut['content_type'] = lut.uri_unsh_no_clean.map(curl.drop_duplicates(subset='url').set_index('url').content_type)
lut.head().T

,0,1,2,3,4
lang,zh,zh,zh,zh,zh
source,wikipedia,wikipedia,wikipedia,wikipedia,wikipedia
timestamp,2018-08-22 13:21:14,2018-06-25 15:15:42,2017-11-10 21:53:43,2017-11-10 21:53:43,2017-11-10 21:53:43
uri,https://web.archive.org/web/20150626164255/htt...,https://web.archive.org/web/20140727005451/htt...,https://archive.is/20140828023420/http://www.w...,https://web.archive.org/web/20140803200140/htt...,https://web.archive.org/web/20140808044248/htt...
uri_cln,web.archive.org/web/20150626164255/http://www....,web.archive.org/web/20140727005451/http://trav...,archive.is/20140828023420/http://www.worldjour...,web.archive.org/web/20140803200140/http://www....,web.archive.org/web/20140808044248/http://m.al...
tld,archive.org,archive.org,archive.is,archive.org,archive.org
is_shortened,False,False,False,False,False
uri_unshrtn,NaN,NaN,NaN,NaN,NaN
uri_cln_unshrtn,web.archive.org/web/20150626164255/http://www....,web.archive.org/web/20140727005451/http://trav...,archive.is/20140828023420/http://www.worldjour...,web.archive.org/web/20140803200140/http://www....,web.archive.org/web/20140808044248/http://m.al...
uri_unshrtn_bitly,NaN,NaN,NaN,NaN,NaN


In [5]:
import numpy as np

In [7]:
lut['archived'] = lut[['uri_cln_wb_avail', 'uri_cln_unshrtn_wb_avail', 'uri_mem_avail', 'uri_unsh_no_clean_mem_avail']].any(axis=1)


In [8]:
def get_values(g):
    return ','.join(list(g.unique()))
tlds_to_remove = ['fb.me', 'facebook.com', 'youtu.be', 'youtube.com']
uris_for_appr = lut[ (~lut.tld_unshrtn.isin(tlds_to_remove))].groupby(['uri_unsh_no_clean', 'tld_unshrtn', 'status_code']).agg({'archived': np.max, 'source': get_values}).reset_index()

In [11]:
uris_for_appr.groupby('source').size()

source
google                       14041
twitter                      37486
twitter,google                 353
wikipedia                     6697
wikipedia,google                16
wikipedia,twitter              343
wikipedia,twitter,google        11
wikipedia,youtube               11
wikipedia,youtube,twitter        8
youtube                       1817
youtube,google                   1
youtube,twitter                 24
dtype: int64

In [50]:
lut[(~lut.lang.str.contains('sco').fillna(False)) & (lut.uri_unsh_no_clean.isin(uris_for_appr[(uris_for_appr.source=='wikipedia,twitter,google')].uri_unsh_no_clean))].groupby('uri_unsh_no_clean').lang.nunique().sort_values(ascending=False)

uri_unsh_no_clean
https://en.wikipedia.org/wiki/2014_Israel%E2%80%93Gaza_conflict                                                                      3
https://www.ynetnews.com/articles/0,7340,L-4564678,00.html                                                                           2
https://www.nytimes.com/interactive/2014/07/15/world/middleeast/toll-israel-gaza-conflict.html                                       2
http://www.newsru.co.il/mideast/08jul2014/day2_101.html                                                                              2
https://www.ynet.co.il/articles/0,7340,L-4558859,00.html                                                                             1
https://www.ynet.co.il/articles/0,7340,L-4555306,00.html                                                                             1
https://www.nytimes.com/2014/08/02/world/europe/anger-in-europe-over-the-israeli-gaza-conflict-reverberates-as-anti-semitism.html    1
http://www.newsru.co.il/photo/13jul20

In [45]:
pd.set_option('display.max_colwidth', -1)

In [51]:
url_example = 'https://www.ynetnews.com/articles/0,7340,L-4564678,00.html'
lut[(~lut.lang.str.contains('sco').fillna(False)) &
    (lut.uri_unsh_no_clean==url_example)][['lang','uri', 'source','uri_unsh_no_clean', 'timestamp','archived']]

,lang,uri,source,uri_unsh_no_clean,timestamp,archived
4061,fr,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",wikipedia,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2017-12-12 04:15:40,True
53656,de,http://bit.ly/1zD8Tyw,twitter,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-27 12:08:00,True
53657,de,http://bit.ly/1zD8Tyw,twitter,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-27 12:08:00,True
53659,de,http://ift.tt/1pgeSsV,twitter,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-27 12:05:00,True
53664,de,http://ift.tt/1pgeSsV,twitter,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-27 11:58:00,True
53920,de,http://fb.me/6TdfxV4dJ,twitter,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-28 06:12:00,True
54710,de,http://ift.tt/1pgeSsV,twitter,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-09-03 06:44:00,True
8380,de,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",google,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-27 23:18:29,True
15552,de,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",google,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-27 23:18:29,True
63112,de,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",google,"https://www.ynetnews.com/articles/0,7340,L-4564678,00.html",2014-08-27 23:18:29,True


In [16]:
uris_for_appr[(uris_for_appr.source=='twitter,google') & (uris_for_appr.tld_unshrtn=='embassies.gov.il')].iat[0, 0]

'http://embassies.gov.il/houston/News/CurrentAffairs/Pages/Operation-Protective-Edge-QA.aspx'

In [17]:
lut[lut.uri_unsh_no_clean=='http://embassies.gov.il/houston/News/CurrentAffairs/Pages/Operation-Protective-Edge-QA.aspx']

,lang,source,timestamp,uri,uri_cln,tld,is_shortened,uri_unshrtn,uri_cln_unshrtn,uri_unshrtn_bitly,uri_unshrtn_raw,tld_unshrtn,uri_cln_wb_avail,uri_cln_unshrtn_wb_avail,uri_unsh_no_clean,uri_mem_avail,uri_unsh_no_clean_mem_avail,status_code,content_type,archived
38592,de,twitter,2014-07-16 16:45:00,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False
8515,de,google,NaT,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False
63271,de,google,NaT,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False
70405,de,google,NaT,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False
108213,de,google,NaT,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False
109963,de,google,NaT,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False
124585,de,google,NaT,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False
174218,de,google,NaT,http://embassies.gov.il/houston/News/CurrentAf...,embassies.gov.il/houston/News/CurrentAffairs/P...,embassies.gov.il,False,NaN,embassies.gov.il/houston/News/CurrentAffairs/P...,NaN,NaN,embassies.gov.il,False,False,http://embassies.gov.il/houston/News/CurrentAf...,False,False,307.0,NaN,False


In [54]:
lut.groupby('archived').uri_unsh_no_clean.nunique()/lut.uri_unsh_no_clean.nunique()

archived
False    0.617462
True     0.382790
Name: uri_unsh_no_clean, dtype: float64

In [55]:
lut.groupby('archived').uri_cln.nunique()/lut.uri_cln.nunique()

archived
False    0.587031
True     0.412969
Name: uri_cln, dtype: float64

In [79]:
lut['is_unshrtn'] = lut.uri!=lut.uri_unsh_no_clean

In [80]:
simp = lut.groupby(['uri_unsh_no_clean', 'tld_unshrtn', 'status_code']).agg({'archived': np.max, 'source': get_values, 'is_shortened': np.max,  'is_unshrtn': np.max}).reset_index()
simp.head()

,uri_unsh_no_clean,tld_unshrtn,status_code,archived,source,is_shortened,is_unshrtn
0,//tools.wmflabs.org/geohack/geohack.php?language=ar&pagename=%D8%A7%D9%84%D8%AD%D8%B1%D8%A8_%D8%B9%D9%84%D9%89_%D8%BA%D8%B2%D8%A9_2014&params=30.6667_N_34.8333_E_globe:earth_type:landmark,wmflabs.org,0.0,False,wikipedia,False,False
1,//tools.wmflabs.org/geohack/geohack.php?language=ar&pagename=%D8%A7%D9%84%D8%AD%D8%B1%D8%A8_%D8%B9%D9%84%D9%89_%D8%BA%D8%B2%D8%A9_2014&params=30_40_N_34_50_E_,wmflabs.org,0.0,False,wikipedia,False,False
2,//tools.wmflabs.org/geohack/geohack.php?language=bn&pagename=%E0%A7%A8%E0%A7%A6%E0%A7%A7%E0%A7%AA_%E0%A6%87%E0%A6%B8%E0%A6%B0%E0%A6%BE%E0%A6%AF%E0%A6%BC%E0%A7%87%E0%A6%B2%E0%A7%87%E0%A6%B0_%E0%A6%AB%E0%A6%BF%E0%A6%B2%E0%A6%BF%E0%A6%B8%E0%A7%8D%E0%A6%A4%E0%A6%BF%E0%A6%A8_%E0%A6%86%E0%A6%95%E0%A7%8D%E0%A6%B0%E0%A6%AE%E0%A6%A3&params=30_40_N_34_50_E_,wmflabs.org,0.0,False,wikipedia,False,False
3,//tools.wmflabs.org/geohack/geohack.php?language=fa&pagename=%D9%86%D8%A8%D8%B1%D8%AF_%DB%B2%DB%B0%DB%B1%DB%B4_%D8%A7%D8%B3%D8%B1%D8%A7%D8%A6%DB%8C%D9%84_%D9%88_%D8%BA%D8%B2%D9%87&params=30_40_N_34_50_E_,wmflabs.org,0.0,False,wikipedia,False,False
4,//tools.wmflabs.org/geohack/geohack.php?language=id&pagename=Konflik_Israel%E2%80%93Gaza_2014&params=30_40_N_34_50_E_,wmflabs.org,0.0,False,wikipedia,False,False


In [81]:
simp[simp.source.str.contains(',')].shape[0] / simp.shape[0]

0.012215568862275449

In [82]:
scl = lut.groupby(['uri_cln_unshrtn', 'tld_unshrtn', 'status_code']).agg({'archived': np.max, 'source': get_values, 'is_shortened': np.max}).reset_index()
scl[scl.source.str.contains(',')].shape[0] / scl.shape[0]

0.012497742459815785

In [83]:
simp.archived.value_counts() / simp.shape[0]

False    0.616826
True     0.383174
Name: archived, dtype: float64

In [84]:
scl.archived.value_counts() / scl.shape[0]

False    0.61906
True     0.38094
Name: archived, dtype: float64

In [85]:
simp.archived.value_counts() / simp.shape[0]

False    0.616826
True     0.383174
Name: archived, dtype: float64

In [86]:
simp[simp.is_shortened].archived.value_counts() / simp[simp.is_shortened].shape[0]

False    0.802531
True     0.197469
Name: archived, dtype: float64

In [87]:
simp[simp.is_unshrtn].archived.value_counts() / simp[simp.is_unshrtn].shape[0]

False    0.798917
True     0.201083
Name: archived, dtype: float64

In [88]:
simp[~simp.is_shortened].archived.value_counts() / simp[~simp.is_shortened].shape[0]

True     0.509073
False    0.490927
Name: archived, dtype: float64

In [90]:
simp[(simp.is_shortened) & (~simp.is_unshrtn)].archived.value_counts() / simp[(simp.is_shortened) & (~simp.is_unshrtn)].shape[0]

False    0.975398
True     0.024602
Name: archived, dtype: float64